In [1]:
%%capture
"""
-----------------------------------------------------------------------------------------------------------------------------------------------------------
This file is used to compute fit a pair of coupled 1D sinusoidal ODEs. It does so using a MSE loss and whole batch methods.
It calculated the Hessian during this process using the library-function approach.
Hessian calculations were considered too lengthy to be part of the same file, and can instead be found in the file
mse_hessians_sinusoidal.ipynb.
-----------------------------------------------------------------------------------------------------------------------------------------------------------
"""

In [2]:
%%capture
%%bash 
pip install torchdiffeq

In [3]:
import os
import argparse
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--method', type=str, choices=['dopri5', 'adams'], default='dopri5')
parser.add_argument('--data_size', type=int, default=1000)
parser.add_argument('--batch_time', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=20)
parser.add_argument('--niters', type=int, default=100)
parser.add_argument('--test_freq', type=int, default=20)
parser.add_argument('--viz', action='store_true')
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--adjoint', action='store_true')
args = parser.parse_args(args=[])

args.niters = 1500
args.test_freq = 20
args.viz = True
args.method = 'dopri5'
args.data_size = 50

In [5]:
#The technique only works when the adjoint method is not used. If it is used, the Hessian returned is a matrix of zeros.
adjoint = False

if adjoint == True:
    from torchdiffeq import odeint_adjoint as odeint
if adjoint == False:
    from torchdiffeq import odeint

device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

true_y0 = torch.tensor([[2., 1.]]).to(device)
t = torch.linspace(0., 7., args.data_size).to(device)
f = 1.0
true_A = torch.tensor([[0.0, f], [-f, -0.0]]).to(device)

class Lambda(nn.Module):

    def forward(self, t, y):
        return torch.mm(y, true_A)

#The true solution defines two sinusoidal curves.
with torch.no_grad():
    true_y = odeint(Lambda(), true_y0, t, method = args.method)

In [6]:
%%capture
def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        
if args.viz:
    makedirs('png')
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(12, 4), facecolor='white')    

In [7]:
def visualize(true_y, pred_y, odefunc, itr):

  """
  This slightly altered version of the function visualize() seems to work fine. The only change is that I have moved the plt.figure() part of the code
  inside the function itself, i.e. I am creating a new figure environment for every figure, instead of editing the same environment multiple times.
  """

  if args.viz:

    fig = plt.figure(figsize=(12, 4), facecolor='white')
    ax_traj = fig.add_subplot(131, frameon=False)        
    ax_phase = fig.add_subplot(132, frameon=False)
    ax_vecfield = fig.add_subplot(133, frameon=False)

    ax_traj.set_title('Trajectories')
    ax_traj.set_xlabel('t')
    ax_traj.set_ylabel('x,y')
    ax_traj.plot(t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 0], t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 1], 'g-')
    ax_traj.plot(t.cpu().numpy(), pred_y.cpu().numpy()[:, 0, 0], '--', t.cpu().numpy(), pred_y.cpu().numpy()[:, 0, 1], 'b--')
    ax_traj.set_xlim(t.cpu().min(), t.cpu().max())
    ax_traj.set_ylim(-2.5, 2.5)

    ax_phase.set_title('Phase Portrait')
    ax_phase.set_xlabel('x')
    ax_phase.set_ylabel('y')
    ax_phase.plot(true_y.cpu().numpy()[:, 0, 0], true_y.cpu().numpy()[:, 0, 1], 'g-')
    ax_phase.plot(pred_y.cpu().numpy()[:, 0, 0], pred_y.cpu().numpy()[:, 0, 1], 'b--')
    ax_phase.set_xlim(-2.5, 2.5)
    ax_phase.set_ylim(-2.5, 2.5)

    ax_vecfield.set_title('Learned Vector Field')
    ax_vecfield.set_xlabel('x')
    ax_vecfield.set_ylabel('y')

    y, x = np.mgrid[-2:2:21j, -2:2:21j]
    dydt = odefunc(0, torch.Tensor(np.stack([x, y], -1).reshape(21 * 21, 2)).to(device)).cpu().detach().numpy()
    mag = np.sqrt(dydt[:, 0]**2 + dydt[:, 1]**2).reshape(-1, 1)
    dydt = (dydt / mag)
    dydt = dydt.reshape(21, 21, 2)

    ax_vecfield.streamplot(x, y, dydt[:, :, 0], dydt[:, :, 1], color="black")
    ax_vecfield.set_xlim(-2, 2)
    ax_vecfield.set_ylim(-2, 2)

    fig.tight_layout()
    plt.savefig('png/{:03d}'.format(itr))
    plt.draw()
    plt.pause(0.001)
    plt.close()

In [8]:
class ODEFunc(nn.Module):
    """
    Defines a very simple neural net.
    """

    def __init__(self):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(2, 8),
            nn.Softplus(),
            nn.Linear(8, 2),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        return self.net(y)

In [ ]:
if __name__ == '__main__':
    """
    Executes the programme. This includes doing the following:

      - Trains the network;
      - Outputs the results in a series of png files (if desired);
      - Outputs hessian matrix information in list form.
    """

    ii = 0

    func = ODEFunc().to(device)
    
    optimizer = optim.RMSprop(func.parameters(), lr=1e-3) 
    loss_data = []

    loss_func = nn.MSELoss()

    for itr in range(1, args.niters + 1):
  
        optimizer.zero_grad()                                 
                    
        pred_y = odeint(func, true_y0, t, method = args.method)
        loss = loss_func(pred_y, true_y)     
        loss.backward() 
      
        if itr % args.test_freq == 0 or itr == 1:
          ii += 1       
          with torch.no_grad():
              loss_data.append((itr, loss.item()))
              print('Iter {:04d} | Total Loss {:.6f}'.format(itr, loss.item()))
              visualize(true_y, pred_y, func, ii)
              #torch.save(func,  '/content/drive/MyDrive/colab_notebooks/NODE_Hessian/calculating_hessians'
                               # + '/testing_on_simple_nodes/sinusoidal_curve/mse_loss'
                               # + '/08_dims/models/model_' + str(itr) + '.pt')       

        optimizer.step()                                                                   
        print('\rIteration: ' + str(itr), end = '')

In [41]:
torch.save(loss_data, '/content/drive/MyDrive/colab_notebooks/NODE_Hessian/calculating_hessians'
                      + '/testing_on_simple_nodes/sinusoidal_curve/mse_loss'
                      + '/08_dims/loss_data.pt')

In [ ]:
#Create a plot of the loss curve.
itrs = []
data = []

for item in loss_data:
  itrs.append(item[0])
  data.append(item[1])

plt.figure(figsize=(10,8))
plt.rcParams.update({'font.size': 14})
plt.plot(itrs, data)
plt.title('Loss function\nGradient Descent'
          + '\nLearning Rate = 1e-3' )
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.savefig('/content/drive/MyDrive/colab_notebooks/NODE_Hessian/calculating_hessians'
            + '/testing_on_simple_nodes/sinusoidal_curve/mse_loss'
            + '/08_dims/loss_curve.png')
plt.show()